<a href="https://colab.research.google.com/github/alik604/cyber-security/blob/master/Intrusion-Detection/KDD%20cup%20'99/Intrusion%20Detection%20V2%2099.99acc%20Multiclass_CLF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intrusion Detection 

### Dataset from https://www.kaggle.com/what0919/intrusion-detection

### Sample code used: https://www.kaggle.com/nidhirastogi/intrusion-detection/data


In [1]:
%%capture

%config IPCompleter.greedy=True
import pandas as pd
# import seaborn as sns
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# !pip install xgboost
# !pip install lightgbm
# !pip install catboost
import xgboost  # xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier
import lightgbm # lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html
import catboost # catboost.ai/docs/concepts/python-quickstart.html



import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")

In [2]:
# %%bash

# # wget dataset
# # wget "https://github.com/alik604/cyber-security/raw/master/Intrusion-Detection/KDD%20cup%20'99/kddcup.data.7z"
# # repeat after me, GitHub is a Content Delivery Network

# # unzip local kddcup.data.7z
# sudo apt-get install p7zip-full
# 7z x kddcup.data.7z

# ls

In [3]:
columns = ["duration" , "protocol_type" , "service" , "flag" , "src_bytes" , "dst_bytes" , "land" , "wrong_fragment" , "urgent" , "hot" , "num_failed_logins" , "logged_in" , "num_compromised" ,
           "root_shell" , "su_attempted" , "num_root" , "num_file_creations" , "num_shells" , "num_access_files" , "num_outbound_cmds" , "is_host_login" , "is_guest_login" , "count" , "srv_count" ,
           "serror_rate" , "srv_serror_rate" , "rerror_rate" , "srv_rerror_rate" , "same_srv_rate" , "diff_srv_rate" , "srv_diff_host_rate" , "dst_host_count" , "dst_host_srv_count" , "dst_host_same_srv_rate" ,
           "dst_host_diff_srv_rate" , "dst_host_same_src_port_rate" , "dst_host_srv_diff_host_rate" , "dst_host_serror_rate" , "dst_host_srv_serror_rate" , "dst_host_rerror_rate" , "dst_host_srv_rerror_rate" , "xAttack"]

all = pd.read_csv('./kddcup.data.csv', names=columns) # index_col=0

all.head(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,xAttack
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.


In [4]:
le = preprocessing.LabelEncoder()

print(set(np.unique(all['xAttack'])), end="\n\n")
print(Counter(all['xAttack']))

for col in ['protocol_type', 'service', 'flag', 'xAttack']:
  all[col] = le.fit_transform(all[col])

all.head(5)

{'guess_passwd.', 'portsweep.', 'spy.', 'land.', 'warezmaster.', 'loadmodule.', 'phf.', 'nmap.', 'satan.', 'normal.', 'smurf.', 'rootkit.', 'back.', 'teardrop.', 'ipsweep.', 'imap.', 'multihop.', 'ftp_write.', 'buffer_overflow.', 'warezclient.', 'perl.', 'pod.', 'neptune.'}

Counter({'smurf.': 2807886, 'neptune.': 1072017, 'normal.': 972781, 'satan.': 15892, 'ipsweep.': 12481, 'portsweep.': 10413, 'nmap.': 2316, 'back.': 2203, 'warezclient.': 1020, 'teardrop.': 979, 'pod.': 264, 'guess_passwd.': 53, 'buffer_overflow.': 30, 'land.': 21, 'warezmaster.': 20, 'imap.': 12, 'rootkit.': 10, 'loadmodule.': 9, 'ftp_write.': 8, 'multihop.': 7, 'phf.': 4, 'perl.': 3, 'spy.': 2})


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,xAttack
0,0,1,24,9,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,11
1,0,1,24,9,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,11
2,0,1,24,9,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,11
3,0,1,24,9,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,11
4,0,1,24,9,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,11


### Option to `USE_HALF` of the data for faster development

In [5]:
USE_HALF = True

if USE_HALF:
    X_train, X_test, y_train, y_test = train_test_split(all.drop(['xAttack'], axis=1), all['xAttack'], test_size=0.5, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

else: 
    X_train, X_test, y_train, y_test = train_test_split(all.drop(['xAttack'], axis=1), all['xAttack'], test_size=0.2, random_state=42)

#### Drop features with lowest corr


In [6]:
# print('\n')
tmp = all.corr().abs()#.sort_values('xAttack')
garbage_corr = tmp['xAttack'].nsmallest(5)
garbage_corr
garbage_corr = garbage_corr.index.to_list() # list(garbage.index)

src_bytes          0.000315
is_host_login      0.000565
dst_bytes          0.000789
urgent             0.001129
num_compromised    0.002170
Name: xAttack, dtype: float64

#### Drop features with lowest STD

In [7]:
tmp = all.std(axis=0).to_frame()
garbage_std = tmp.nsmallest(5, columns=0)
garbage_std
garbage_std = garbage_std.index.to_list()
print(set(garbage_corr + garbage_std))

,0
num_outbound_cmds,0.000000
is_host_login,0.000639
land,0.002391
urgent,0.007215
num_failed_logins,0.007299


{'src_bytes', 'num_failed_logins', 'num_compromised', 'urgent', 'dst_bytes', 'is_host_login', 'num_outbound_cmds', 'land'}


### Option to remove features with lowest STD and lowest corr

In [8]:
REMOVE_FEATURES = True

if REMOVE_FEATURES and garbage_std[0] in X_train.columns: 
    to_remove = set(garbage_corr + garbage_std)
    X_train.drop(to_remove, axis=1, inplace=True)
    X_test.drop(to_remove, axis=1, inplace=True)


## Do some ML...

In [9]:
from sklearn import linear_model

from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import IsolationForest

from sklearn.metrics import classification_report, f1_score


from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [10]:
## runs very long... so i set `N`
N = 500000
LR = linear_model.LogisticRegression()
LR.fit(X_train[:N], y_train[:N])
lr_score = LR.score(X_test[:N], y_test[:N])
print('Logistic regression processing ,,,')
print('Logistic regression Score: %.2f %%' % lr_score)

print(classification_report(y_test, LR.predict(X_test)))

LogisticRegression()

Logistic regression processing ,,,
Logistic regression Score: 0.99 %
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       229
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         4
           5       0.00      0.00      0.00      1201
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         3
           9       0.99      0.99      0.99    107152
          10       0.00      0.00      0.00       222
          11       0.96      0.99      0.98     97012
          13       0.00      0.00      0.00         3
          14       0.00      0.00      0.00        26
          15       0.29      0.00      0.00      1020
          16       0.00      0.00      0.00         2
          17       0.96      0.82      0.88      1603
          18       1.00      1.00      1.00    281163
          20

In [11]:
AB = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5), n_estimators=40) # 50
RF = RandomForestClassifier(n_estimators=100, criterion='entropy')
ET = ExtraTreesClassifier(n_estimators=100, criterion='gini', bootstrap=False)
GB = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=200, max_features='auto')

In [12]:
_ = AB.fit(X_train[:80000], y_train[:80000])
AB_feature = AB.feature_importances_
ab_score = AB.score(X_test, y_test)

print('AdaBoostClassifier processing...')
print('AdaBoostClassifier Score: %.3f %%' % ab_score)
# print(classification_report(y_test, AB.predict(X_test)))

AdaBoostClassifier processing...
AdaBoostClassifier Score: 0.999 %


In [13]:
_ = RF.fit(X_train, y_train)
RF_feature = RF.feature_importances_
rf_score = RF.score(X_test, y_test)

print('RandomForestClassifier processing...')
print('RandomForestClassifier Score: %.3f %%' % rf_score)

RandomForestClassifier processing...
RandomForestClassifier Score: 1.000 %


In [14]:
_ = ET.fit(X_train, y_train)
ET_feature = ET.feature_importances_
et_score = ET.score(X_test, y_test)

print('ExtraTreesClassifier processing...')
print('ExtraTreeClassifier: %.3f %%' % et_score)

ExtraTreesClassifier processing...
ExtraTreeClassifier: 1.000 %


In [15]:
_ = GB.fit(X_train[:40000], y_train[:40000])
GB_feature = GB.feature_importances_
gb_score = GB.score(X_test, y_test)

print('GradientBoostingClassifier processing...')
print('GradientBoostingClassifier Score: %.3f %%' % gb_score)

GradientBoostingClassifier processing...
GradientBoostingClassifier Score: 0.989 %


In [16]:
feature_df = pd.DataFrame({'features': X_train.columns.values, # names
                           'AdaBoost' : AB_feature,
                           'RandomForest' : RF_feature,
                           'ExtraTree' : ET_feature,
                           'GradientBoost' : GB_feature
                          })
#feature_df.features
feature_df.head(10)

,features,AdaBoost,RandomForest,ExtraTree,GradientBoost
0,duration,7.968892e-05,8.174294e-04,0.000168,0.003402
1,protocol_type,4.479195e-02,8.812622e-02,0.103234,0.007594
2,service,1.037501e-01,1.027312e-01,0.024460,0.030948
3,flag,5.254557e-02,4.812925e-02,0.032790,0.008353
4,wrong_fragment,1.021569e-01,7.150973e-04,0.000573,0.000058
5,hot,1.901631e-02,1.629530e-03,0.001030,0.001099
6,logged_in,3.773389e-02,1.530483e-02,0.056275,0.002932
7,root_shell,5.030321e-02,8.848849e-06,0.000009,0.000010
8,su_attempted,0.000000e+00,2.977811e-07,0.000001,0.000000
9,num_root,9.716080e-09,9.638793e-06,0.000008,0.012764


In [17]:
TOP_N_FEATURES = 20

n = 30
a_f = feature_df.nlargest(n, 'AdaBoost')
e_f = feature_df.nlargest(n, 'ExtraTree')
g_f = feature_df.nlargest(n, 'GradientBoost')
r_f = feature_df.nlargest(n, 'RandomForest')

# concatenate results
result = pd.concat([a_f, e_f, g_f, r_f], axis=0)
result.set_index("features", inplace=True)

# 80 rows - > n rows
result = result.groupby('features').sum()

# get best featues via voting
result['Sum'] = result.sum(axis=1)
result.sort_values('Sum', ascending=False,inplace=True)
result[:TOP_N_FEATURES]

# get names of best features
top_features = result[:TOP_N_FEATURES].index.to_list()
top_features

,AdaBoost,RandomForest,ExtraTree,GradientBoost,Sum
features,,,,,
count,1.920625e-01,0.694683,0.514748,0.933194,2.334688
srv_count,2.646765e-01,0.605307,0.540050,0.204992,1.615026
same_srv_rate,2.061557e-01,0.214481,0.347402,0.534401,1.302440
dst_host_diff_srv_rate,2.419626e-01,0.159284,0.049250,0.771300,1.221797
service,4.150005e-01,0.410925,0.097840,0.123793,1.047559
dst_host_same_src_port_rate,1.248629e-01,0.325353,0.383798,0.149884,0.983898
protocol_type,1.791678e-01,0.352505,0.412936,0.030378,0.974986
dst_host_srv_diff_host_rate,6.003481e-02,0.022051,0.014583,0.876946,0.973614
dst_host_srv_count,5.485100e-01,0.132385,0.266100,0.022885,0.969880


['count',
 'srv_count',
 'same_srv_rate',
 'dst_host_diff_srv_rate',
 'service',
 'dst_host_same_src_port_rate',
 'protocol_type',
 'dst_host_srv_diff_host_rate',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'flag',
 'logged_in',
 'diff_srv_rate',
 'dst_host_serror_rate',
 'wrong_fragment',
 'dst_host_count',
 'dst_host_srv_serror_rate',
 'root_shell',
 'srv_serror_rate',
 'serror_rate']

### Remove `selected_features` from our Data

In [18]:
X_train_SF = X_train[top_features]
X_test_SF = X_test[top_features]


assert X_train_SF.shape[0] == y_train.shape[0]

# numb_of_train_points = 410000

In [26]:
DTC = DecisionTreeClassifier()
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=150, random_state=42, n_jobs=-1)
XGB = DecisionTreeClassifier(random_state=42) # xgboost.XGBClassifier(n_estimators=100, n_jobs=-1)
GBM = lightgbm.LGBMClassifier(objective='multiclass', n_estimators= 400)

list_of_CLFs_names = []
list_of_CLFs = [DTC, RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train, y_train)
    pred = clf.score(X_test, y_test)
    pred = f1_score(y_test, clf.predict(X_test), average='macro')
    name = str(type(clf)).split(".")[-1][:-2]
    print("F1: %0.5f for the %s" % (pred, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

F1: 0.66420 for the DecisionTreeClassifier
F1: 0.69469 for the RandomForestClassifier
F1: 0.69603 for the ExtraTreesClassifier
F1: 0.70161 for the DecisionTreeClassifier
F1: 0.14775 for the LGBMClassifier


In [23]:
DTC = DecisionTreeClassifier(random_state=42)
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
XGB = DecisionTreeClassifier(random_state=42) # xgboost.XGBClassifier(n_estimators=100, n_jobs=-1)
GBM = lightgbm.LGBMClassifier(objective='multiclass', n_estimators= 400)

list_of_CLFs_names = []
list_of_CLFs = [DTC, RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_SF, y_train)
    # pred = clf.score(X_test_SF, y_test)
    pred = f1_score(y_test, clf.predict(X_test_SF), average='macro')
    name = str(type(clf)).split(".")[-1][:-2]
    print("F1: %0.5f for the %s" % (pred, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

F1: 0.55387 for the DecisionTreeClassifier
F1: 0.64427 for the RandomForestClassifier
F1: 0.63751 for the ExtraTreesClassifier
F1: 0.55387 for the DecisionTreeClassifier
F1: 0.14829 for the LGBMClassifier


In [24]:
from sklearn.decomposition import * 
wanted_explained_variance_ratio = 0.9999
pca = PCA(wanted_explained_variance_ratio)
_ = pca.fit(X_train)
X_train_PCA = pca.transform(X_train)
X_test_PCA = pca.transform(X_test)

X_train.shape
X_train_PCA.shape

(1959372, 33)

(1959372, 5)

In [25]:
DTC = DecisionTreeClassifier()
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
XGB = DecisionTreeClassifier(random_state=42) # xgboost.XGBClassifier(n_estimators=100, n_jobs=-1)
GBM = lightgbm.LGBMClassifier(objective='multiclass', n_estimators= 400)

list_of_CLFs_names = []
list_of_CLFs = [DTC, RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_PCA, y_train)
    # pred = clf.score(X_test_SF, y_test)
    pred = f1_score(y_test, clf.predict(X_test_PCA), average='macro')
    name = str(type(clf)).split(".")[-1][:-2]
    print("F1: %0.5f for the %s" % (pred, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

F1: 0.41645 for the DecisionTreeClassifier
F1: 0.46076 for the RandomForestClassifier
F1: 0.46275 for the ExtraTreesClassifier
F1: 0.43743 for the DecisionTreeClassifier
F1: 0.08952 for the LGBMClassifier


## try oversampling the data

In [31]:
# //  USING, X_train_PCA, X_test_PCA

# %pip install imblearn

X_train.shape
X_train_OS.shape

(1959372, 33)

(24707760, 5)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
print(sorted(Counter(y_train).items()))

ros = RandomOverSampler(random_state=0)
X_train_OS, y_train_OS = ros.fit_resample(X_train_SF, y_train)
## DO NOT DO. thankfully it didnt run, this is now allowable
# X_test_OS, y_test_OS = ros.resample(X_test_PCA, y_test)

print(sorted(Counter(y_train_OS).items()))

In [ ]:
# caution, very slow
DTC = DecisionTreeClassifier()
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
XGB = DecisionTreeClassifier(random_state=42) # xgboost.XGBClassifier(n_estimators=100, n_jobs=-1)
GBM = lightgbm.LGBMClassifier(objective='multiclass', n_estimators= 400)

list_of_CLFs_names = []
list_of_CLFs = [DTC, RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_OS, y_train_OS)
    # pred = clf.score(X_test_SF, y_test)
    pred = f1_score(y_test, clf.predict(X_test), average='macro')
    name = str(type(clf)).split(".")[-1][:-2]
    print("F1: %0.5f for the %s" % (pred, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)